In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [ ]:
plt.rcParams = plt.rcParamsOrig

# Direct Sampling

Dari 30 orang yang Anda tunjukkan UI baru aplikasi Anda, 22 orang berkata mereka menyukainya. Apakah ini kebetulan? 📱

## Metode Klasik

$$
P(X \ge k) = 1 - \sum_{c=0}^{k-1} \binom{n}{c} p^c (1-p)^{n-c}
$$

In [ ]:
import scipy.stats as ss

x = np.arange(31)
y = ss.binom(30, 0.5).pmf(x)
plt.bar(x, y)
plt.axvline(22, c='tab:red')
plt.annotate("", xy=(24, 0.08), xytext=(22, 0.08),
             arrowprops=dict(arrowstyle="->", color='tab:red'))
plt.xlabel('$k$')
plt.ylabel('$P(X=k)$');

## Metode Sampling

In [ ]:
# Kode Anda di sini

# Shuffling

Diberikan data alokasi pupuk lama dan baru beserta hasil panennya seperti di bawah ini. Apakah pupuk baru berdampak pada hasil panen yang lebih banyak? 🍅

In [ ]:
x1 = np.array([29.2, 11.4, 25.3, 16.5, 21.1]) # 20.70
x2 = np.array([26.6, 23.7, 28.5, 14.2, 17.9, 24.3]) # 22.53
n1 = len(x1)
n2 = len(x2)

## Metode Klasik

$$
t = \frac{\bar X_1 - \bar X_2}{\sqrt{\frac{s_1^2}{n_1} + \frac{s_2^2}{n_2}}}
$$

In [ ]:
t = np.round(
    (x2.mean() - x1.mean()) / np.sqrt(np.var(x1, ddof=1)/n1 + np.var(x2, ddof=1)/n2),
    3
)

$$
\nu \approx \frac{\left(\frac{s_1^2}{N_1} + \frac{s_2^2}{N_2}\right)^2}{\frac{s_1^4}{N_1^2(N_1-1)} + \frac{s_2^4}{N_2^2(N_2-1)}}
$$

In [ ]:
num = (np.var(x1, ddof=1)/n1 + np.var(x2, ddof=1)/n2) ** 2
denom = np.var(x1, ddof=1)**2/(n1**2 * (n1 - 1)) + np.var(x2, ddof=1)**2/(n2**2 * (n2 - 1))
nu = num / denom
ss.t(nu).ppf(1 - 0.05)

## Metode Engineer

In [ ]:
from statsmodels.stats.weightstats import ttest_ind

t, p, dof = ttest_ind(
    x2, x1,
    alternative='larger',
    usevar='unequal'
)
p

## Metode Sampling

In [ ]:
np.random.seed(42)

x = np.array([29.2, 11.4, 26.6, 23.7, 25.3, 28.5, 14.2, 17.9, 16.5, 21.1, 24.3])
diff = []
# Kode Anda di sini

In [ ]:
sns.histplot(x=diff, bins=30, element='step', fill=False)
plt.axvline(1.83, c='tab:red')
plt.annotate("", xy=(3, 400), xytext=(1.83, 400),
             arrowprops=dict(arrowstyle="->", color='tab:red'))
plt.xlabel('$X$')
plt.ylabel('freq');

# Bootstrapping

Seorang pengemudi ojek online mendapatkan order tiap harinya selama 21 hari sebagai berikut. Seberapa yakin dia dengan rata-rata jumlah order per harinya? 🛵

In [ ]:
np.random.seed(42)

x = np.random.poisson(20, size=(3, 7))
sns.heatmap(
    x,
    square=True,
    cbar=False,
    annot=True,
    fmt='d',
    cmap='Greens'
)
plt.xticks([])
plt.yticks([]);

## Metode Klasik

$$
\bar{X} = \frac{1}{N} \sum_{i=1}^N x_i
$$

$$
\sigma_{\bar{x}} = \frac{1}{\sqrt{N}} \sqrt{\frac{1}{N-1} \sum_{i=1}^N (x_i - \bar{x})^2}
$$

In [ ]:
# Kode Anda di sini

## Metode Sampling

In [ ]:
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

plt.rcParams['animation.html'] = 'html5'
fig, ax = plt.subplots()

def update(frame):
    bg = np.array([0] * 20 + [1])
    np.random.shuffle(bg)
    bg = bg.reshape(3, 7)
    ax.imshow(bg)
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            c = 'k' if bg[i, j] == 1 else 'w'
            plt.text(j-0.1, i+0.1, x[i,j], c=c)
    plt.axis('off')
    return ax

anim = FuncAnimation(fig, update, frames=21, interval=500)
# anim.save('ojek.gif', writer='imagemagick', fps=21)
anim;

In [ ]:
n_trials = 10_000
data = []
# Kode Anda di sini

In [ ]:
sns.histplot(x=data, bins=30)
plt.xlabel('$\\bar x$')
plt.title(f"order = {data.mean():.2f} $\pm$ {data.std():.2f}")
plt.savefig('ojek-hist.png', bbox_inches='tight')

# Bootstrapped Linear Regression

In [ ]:
np.random.seed(1)

y = x.flatten()
hour = y / (1.75) + np.random.normal(scale=0.7, size=21)
fig, ax = plt.subplots(figsize=(7, 7))
sns.regplot(
    x=hour,
    y=y,
    ci=False,
    ax=ax
)
plt.xlabel('jam kerja')
plt.ylabel('order');

In [ ]:
n_trials = 10_000
models = []
for _ in range(n_trials):
    i = np.random.randint(21, size=21)
    reg = ss.linregress(hour[i], y[i])
    models.append((reg.intercept, reg.slope))

In [ ]:
sns.jointplot(
    x='slope',
    y='intercept',
    data=pd.DataFrame(models, columns=['intercept', 'slope']),
    kind='hex'
);

In [ ]:
fig, ax = plt.subplots(figsize=(7, 7))
intercepts, slopes = np.array(models).T
yhat = np.outer(intercepts, np.ones(21)) + np.outer(slopes, hour)
sns.regplot(
    x=hour,
    y=y,
    ci=False,
    ax=ax
)
plt.fill_between(
    hour,
    yhat.mean(axis=0) - yhat.std(axis=0),
    yhat.mean(axis=0) + yhat.std(axis=0),
    alpha=0.5,
    color='tab:orange'
);

# Cross Validation

Bagaimana trend dari pertumbuhan jumlah kasus harian COVID-19 di Indonesia pada 100 hari pertama? 🦠

In [ ]:
# Sumber: Kawal COVID-19
# https://kawalcovid19.id/
df = pd.read_csv('https://docs.google.com/spreadsheets/d/1ma1T9hWbec1pXlwZ89WakRk-OfVUQZsOCFl4FwZxzVw/export?format=csv&gid=387345074')

In [ ]:
data = df['Kasus harian'].str.replace(',', '').astype(int).reset_index()
train_data = data.head(100).copy()

In [ ]:
sns.regplot(
    x='index',
    y='Kasus harian',
    ci=None,
    data=train_data,
    fit_reg=True,
    marker='.',
    order=1, # orde polinomial
    line_kws=dict(
        color='tab:red',
        alpha=0.5
    )
)
plt.xlabel('hari ke-i')
plt.ylabel('jumlah kasus');

In [ ]:
from sklearn.metrics import mean_squared_error

def regress(x, y, degree):
    p = np.polyfit(x, y, degree)
    reg = np.poly1d(p)
    yhat = reg(x)
    return mean_squared_error(y, yhat)

In [ ]:
mses = []
degrees = range(1, 15)
for degree in degrees:
    mses.append(
        regress(train_data['index'], train_data['Kasus harian'], degree)
    )
plt.plot(degrees, np.sqrt(mses))
plt.xlabel('orde polinomial')
plt.ylabel('RMSE');

## Metode Sampling

In [ ]:
train_data['label'] = np.random.randint(0, 2, size=len(train_data))
mask = train_data['label'] == 1

In [ ]:
plt.scatter(
    x='index',
    y='Kasus harian',
    data=train_data,
    c=np.array(['tab:blue', 'tab:red'])[train_data.label],
    marker='.'
)
plt.xlabel('hari ke-i')
plt.ylabel('jumlah kasus');

In [ ]:
fig, ax = plt.subplots(sharex=True, sharey=True, ncols=2)
configs = dict(
    x='index',
    y='Kasus harian',
    ci=None,
    fit_reg=True,
    scatter=False,
    marker='.',
    order=2
)
sns.regplot(
    data=train_data[mask],
    color='tab:blue',
    ax=ax[0],
    **configs
)
sns.regplot(
    data=train_data[~mask],
    color='tab:red',
    ax=ax[1],
    **configs
)
new_configs = dict(
    x='index',
    y='Kasus harian',
    ci=None,
    fit_reg=False,
    scatter=True,
    marker='.'
)
sns.regplot(
    data=train_data[~mask],
    color='tab:red',
    ax=ax[0],
    **new_configs
)
sns.regplot(
    data=train_data[mask],
    color='tab:blue',
    ax=ax[1],
    **new_configs
)
ax[0].set_xlabel('hari ke-i')
ax[0].set_ylabel('jumlah kasus')
ax[1].set_xlabel('hari ke-i')
ax[1].set_ylabel('');

In [ ]:
p = np.polyfit(train_data[~mask]['index'], train_data[~mask]['Kasus harian'], 2)
reg = np.poly1d(p)
y_true = train_data[mask]['Kasus harian']
y_pred = reg(train_data[mask]['index'])
np.sqrt(mean_squared_error(y_true, y_pred))

In [ ]:
def cross_validate(x, y, degree: int, n_fold: int, metric) -> np.ndarray:
    # Kode Anda di sini
    pass

In [ ]:
np.random.seed(42)

mses = []
emses = []
degrees = range(1, 15)
for degree in degrees:
    mses.append(
        regress(train_data['index'], train_data['Kasus harian'], degree)
    )
    emses.append(
        cross_validate(
            train_data['index'], train_data['Kasus harian'],
            degree=degree,
            n_fold=2,
            metric=mean_squared_error
        ).mean()
    )
plt.plot(degrees, np.sqrt(mses))
plt.plot(degrees, np.sqrt(emses))
plt.xlabel('orde polinomial')
plt.ylabel('RMSE')
plt.legend(['RMSE', 'CV-RMSE']);

In [ ]:
p = np.polyfit(train_data['index'], train_data['Kasus harian'], 7)
reg = np.poly1d(p)
ax = plt.gca()
data['Kasus harian'].plot.line(label='aktual', ax=ax)
ax.plot(reg(data['index']), label='prediksi')

axins = ax.inset_axes([0.1, 0.2, 0.5, 0.5])
data['Kasus harian'].head(200).plot.line(label='aktual', ax=axins)
axins.plot(reg(data['index'].head(200)), label='prediksi')
x1, x2, y1, y2 = 0, 200, 0, 4000
axins.set_xlim(x1, x2)
axins.set_ylim(y1, y2)
axins.set_xticklabels('')
axins.set_yticklabels('')
ax.indicate_inset_zoom(axins, edgecolor="black")

plt.legend(['aktual', 'prediksi']);

# Bonus

Seorang penikmat musik sedang shuffle play lagu soundtrack Charlie's Angels terbaru. Di albumnya ada 11 lagu, 6 di antaranya lagu Ariana Grande. Berapa peluangnya didapatkan lagu Ariana Grande tiga kali berturut-turut? (Sumber: [Twitter](https://twitter.com/waribowo_/status/1196722307722444802))